In [5]:
import csv
import datetime
import time
import os

import matplotlib.pyplot as plt
import collections
from openpyxl import load_workbook

import math
import statistics
import numpy as np
import pandas as pd

In [4]:
def get_filename_from_folder(folder_path, ext=""):
    file_name_list = []
    for file_name in os.listdir(folder_path):
        name = os.path.join(folder_path + "/"+ file_name)
        
        # get file extension
        file_ext = os.path.splitext(file_name)[1]
        # if user give ext and the file extension is not the same, pass
        if ext and file_ext != ext:
            continue
        
        # os.rename(name, os.path.join(folder_path + "/"+ file_name[1:]))
    
        file_name_list.append(name)
        
    return sorted(file_name_list)


def filename_process(target, filename_list):
    
    split_file_name = [ x.split("_") for x in filename_list]
    
    for i in split_file_name:
        try:
            ytr = i[1][1:]
            types = i[3]
            start = i[4]
            end = i[5].split(".")[0]
        
            if target == ytr:
                yield ytr, types, start, end
        except:
            pass


def get_file_by_ytr_and_date(ytr, types, start, filename_list):
    file_list = []
    for fytr, ftypes, fstart, fend in filename_process(ytr, filename_list):
        if fytr == ytr and ftypes == types and fstart == start:
            name = f"./yt_/{fytr}_單主題_{ftypes}_{fstart}_{fend}.xlsx"

            file_list.append(name)
    return file_list


def string_to_datetime(s):
    return datetime.datetime.strptime(s, '%Y/%m/%d')


# opview 資料（趨勢 和 好感度）
def get_ytrs_view_and_emotion_by_startdate(ytrs: list, startdate: str, enddate: str, filename_list_op: list):
    
    ytrs_view = []
    ytrs_emotion = []
    
    for ytr in ytrs: # iter each youtuber
        # read all views data
        op_view_data = collections.OrderedDict()
        for date in DATE:
            for filename in get_file_by_ytr_and_date(ytr, '趨勢', date, filename_list_op):
                wb = load_workbook(filename=filename)
                # op_view_data['name'] = ytr
                opview_wb = wb['日趨勢']
                for i in range(2, opview_wb.max_row+1):
                    day = opview_wb.cell(column=1, row=i).value
                    qty = opview_wb.cell(column=2, row=i).value
                    
                    if day >= startdate and day <= enddate:
                        op_view_data[day] = qty
        ytrs_view.append(op_view_data)

        # read all good bad data
        op_view_emotion = collections.OrderedDict(
            {'good': collections.OrderedDict(), 'bad': collections.OrderedDict()})
        for date in DATE:
            for filename in get_file_by_ytr_and_date(ytr, '情緒', date, filename_list_op):
                # opview 資料（情緒）
                wb = load_workbook(filename=filename)
                opview_wb = wb['日情緒趨勢']
                for i in range(2, opview_wb.max_row+1):

                    day = opview_wb.cell(column=1, row=i).value
                    good = opview_wb.cell(column=2, row=i).value
                    bad = opview_wb.cell(column=3, row=i).value
                    total = opview_wb.cell(column=4, row=i).value
                    if day >= startdate and day <= enddate:
                        op_view_emotion["good"][day] = good
                        op_view_emotion["bad"][day] = bad
        ytrs_emotion.append(op_view_emotion)

    return ytrs_view ,ytrs_emotion


# plot views and good/bad emotions
def plot_view_and_emotion(ytrs_view, ytrs_emotion, ytrs, stepsize=1, figsize=(19,4)):
    for view, emotion, name in zip(ytrs_view, ytrs_emotion, ytrs):
        print(name)
        fig, ax1 = plt.subplots(figsize=figsize)
        ax1.plot(view.values(), figure=fig,
                 linewidth=1, c='black', label="opview")
        ax1.plot(emotion["good"].values(), figure=fig,
                 linewidth=1, c='g', label="good")
        ax1.plot(emotion["bad"].values(), figure=fig,
                 linewidth=1, c='r', label="bad")
        plt.xticks(range(0, len(list(view.keys())), stepsize),
                       list(view.keys())[::stepsize], rotation=20)
        ax1.legend()


def get_news_filename_list_by_ytrs(ytrs, data_folder_path, include_news=["ettoday_keywords.csv", "apple_news.csv", "dcard.csv" "ettoday_tag.csv", "ptt.csv"]):
    filename_list_news_all = []

    for ytr in ytrs:
        print(ytr)
        ytr_news = []  # news list for each ytr
        filename_list_news = get_filename_from_folder(
            data_folder_path + ytr)
        for x in filename_list_news:
            if x.split('/')[-1] in include_news:
                ytr_news.append(x)
        filename_list_news_all.append(ytr_news)

    return filename_list_news_all


def get_news_list_by_filename_list(filename_list_news_et):
    news_list = []
    for filename_list in filename_list_news_et:  # iterate ytr
        news = []
        for filename in filename_list:
            with open(filename, encoding="utf-8") as f:
                reader = csv.reader(f)
                for idx, row in enumerate(reader):
                    # Source,Title,Link,Date,Summary,Text,Like

                    if idx == 0 or row == ['Source', 'Title', 'Link', 'Date', 'Summary', 'Text', 'Like']:
                        continue
                    Source = row[0]
                    Title = "".join(row[1].split())
                    Link = row[2]
                    Date = row[3].split(" ")[0].replace('-', '/')
                    Summary = "".join(row[4].split())
                    Text = row[5]

                    news.append({"source": Source, "title": Title, "link": Link,
                                 "date": Date, "summary": Summary, "text": Text})
        news_list.append(news)
    return news_list


def compare_news_list_and_event_list(multiply=3):
    print('單日大於', multiply, '倍過去七天的平均：')

    # event definition
    for view, emotion, name, news in zip(ytrs_view, ytrs_emotion, ytrs, news_list):
        
        print(name)
        # event definition
        data = [list(view.values())[0]] * 7
        data.extend(list(view.values()))

        count = 0
        event_list = []

        for i, d in enumerate(data[7:], start=7):
            seven_day = data[i-7:i]
            seven_day_avg = sum(seven_day)/7
            if d > multiply * seven_day_avg:
                day = list(view.keys())[i-7]
                event_list.append(day)
        print(event_list)

        # find news
        print('來源     事件日期      日期好壞比  新聞日期     新聞日期好壞比    新聞標題')
        for i in news:
            day_add = datetime.timedelta(days=2)
            try:
                news_date = string_to_datetime(i['date'])
                
            except:
                news_date = i['date'].split(" ")[0].replace("-", "/")
                news_date = string_to_datetime(news_date)

            for event in event_list:
                temp_event = event
                temp_view = view
                temp_emotion = emotion
                event_day = string_to_datetime(event)
                if news_date >= event_day and news_date <= event_day + day_add:
                    try:
                        print(i['source'], event_day.strftime('%Y-%m-%d'), ' ',
                            '%.2f' % (round(
                                emotion['good'][event] / emotion['bad'][event], 2)), '    ',
                            news_date.strftime('%Y-%m-%d'), '  '
                            '%.2f' % (round(
                                emotion['good'][i['date']] / emotion['bad'][i['date']], 2)), '       ',
                            i['title'])
                    except:
                        pass


def generate_ytr_name_list(filename="Youtuber名單 - 道歉.csv"):
    ytr_list = []
    with open(filename, 'r') as f:
        rows = csv.reader(f)
        for row in rows:
            if row[1] == "":
                pass
            if row[1] == "頻道(官方帳號)":
                continue
            ytr_name = []
            ytr_name.append(row[1].replace(
                "/", " ").replace("   ", " ").split("\n")[0])

            ytr_name.extend(row[3].replace("、", "\n").split("\n"))
            ytr_list.append(list(dict.fromkeys(ytr_name)))
    return ytr_list
                    


In [18]:
merged_with_bobo = pd.read_csv("../merged_with_bobo.csv")
sorry_event = pd.read_excel("../Youtuber道歉事件.xlsx",
                   sheet_name="bobo new strategy", header=1)
merged_with_bobo.columns, sorry_event.columns

(Index(['期間', '正面情緒', '負面情緒', '正面情緒(前一天)', '正面情緒(前二天)', '正面情緒(後一天)',
        '正面情緒(後二天)', '負面情緒(前一天)', '負面情緒(前二天)', '負面情緒(後一天)', '負面情緒(後二天)',
        '當天正負情緒比', '前一天正負情緒比', '前二天正負情緒比', '後一天正負情緒比', '後二天正負情緒比', '當天情緒比變化量',
        '隔天情緒比變化量', '前一後一情緒變化量', '前二後二情緒變化量', '聲量', 'Youtuber', '前一天聲量',
        '前二天聲量', '後一天聲量', '後二天聲量', '當天聲量變化量', '隔天聲量變化量', '前一後一聲量變化量',
        '前二後二聲量變化量', 'Dcard', 'Dcard內文', 'PTT', 'PTT內文', 'apple_news',
        'apple_news內文', 'ettoday', 'ettoday內文', 'Mobile01', 'Mobile01內文',
        'opview', 'g/b', 'sub', 'category', 'sorry'],
       dtype='object'),
 Index(['youtuber', 'date', '犯錯的類別', '表達目標人物/觀眾的感謝或諒解', '補償', '有沒有道歉', '解釋',
        '引發同情(說自己有病)', '藉口', '犯錯的類別.1', '表達目標人物/觀眾的感謝或諒解.1', '補償.1', '有沒有道歉.1',
        '解釋.1', '引發同情(說自己有病).1', '藉口.1', '犯錯的類別.2', '表達目標人物/觀眾的感謝或諒解.2', '補償.2',
        '有沒有道歉.2', '解釋.2', '引發同情(說自己有病).2', '藉口.2', '犯錯的類別.3',
        '表達目標人物/觀眾的感謝或諒解.3', '補償.3', '有沒有道歉.3', '解釋.3', '引發同情(說自己有病).3', '藉口.3',
        '犯錯的類別.4', '表達目標人物/觀眾的感

In [117]:
sorry_df = sorry_event[["youtuber", "date", '犯錯的類別']].copy()
sorry_df = sorry_df.rename(columns={'youtuber': 'Youtuber', 'date': '道歉日'})
event_df = merged_with_bobo.copy()
sorry_df.loc[:, '道歉日'] = sorry_df['道歉日'].dt.strftime('%Y/%m/%d')


In [144]:
display(sorry_df.head()), display(event_df.head())
print(sorry_df.shape)

,Youtuber,道歉日,犯錯的類別
0,阿神,2020/03/09,c
1,阿滴英文,2020/04/28,a
2,阿滴英文,2019/12/12,c
3,Joeman,2019/09/26,a
4,Joeman,2021/04/28,d


,期間,正面情緒,負面情緒,正面情緒(前一天),正面情緒(前二天),正面情緒(後一天),正面情緒(後二天),負面情緒(前一天),負面情緒(前二天),負面情緒(後一天),...,apple_news內文,ettoday,ettoday內文,Mobile01,Mobile01內文,opview,g/b,sub,category,sorry
0,2020/01/19,429,54,21.0,21.0,24.0,8.0,70.0,38.0,58.0,...,[],[],[],[],[],Dinter,bad,75.7,電玩實況類,True
1,2020/04/05,42,36,13.0,11.0,16.0,63.0,10.0,18.0,11.0,...,[],[],[],[],[],Dinter,bad,75.7,電玩實況類,True
2,2020/05/03,464,161,18.0,17.0,145.0,64.0,34.0,10.0,62.0,...,[],[],[],[],[],Dinter,bad,75.7,電玩實況類,True
3,2020/05/16,242,173,41.0,16.0,56.0,28.0,81.0,72.0,55.0,...,[],[],[],[],[],Dinter,bad,75.7,電玩實況類,True
4,2020/10/09,207,589,13.0,9.0,175.0,90.0,44.0,26.0,561.0,...,[],[],[],[],[],Dinter,bad,75.7,電玩實況類,True


(76, 3)


In [146]:
DAY_DELTA = 10
BEFORE_EVENT_COUNT = 1
final_df = pd.DataFrame()
event_ytrs = event_df['Youtuber'].unique() # merged_with_bobo 裡面所有的人

with open("../sorry_event_match.csv", "w+"):
    for ytr in sorry_df['Youtuber'].unique(): # google sheet 裡面所有的人

        if ytr not in event_ytrs:   # 如果有人不在 merged_with＿bobo，跳過
            continue

        sorry_date_list = sorry_df[sorry_df['Youtuber'] == ytr]['道歉日'].tolist() # list with sorry date

        # iterate each sorry date
        _focus_event_df = event_df[event_df['Youtuber'] == ytr]

        # 每個人自己的道歉日
        for sorry_day in sorry_date_list:
            
            _target_df = _focus_event_df[_focus_event_df['期間']
                                         < sorry_day][-BEFORE_EVENT_COUNT:]
            _target_df["道歉日"] = sorry_day
            
            final_df = pd.concat(
                [final_df, _target_df], axis=0)
            


            
            


In [147]:
final_df.shape

(60, 47)

In [148]:
final_df


,期間,正面情緒,負面情緒,正面情緒(前一天),正面情緒(前二天),正面情緒(後一天),正面情緒(後二天),負面情緒(前一天),負面情緒(前二天),負面情緒(後一天),...,ettoday內文,Mobile01,Mobile01內文,opview,g/b,sub,category,sorry,道歉日,犯錯的類別
4095,2020/02/22,468.0,285.0,85.0,115.0,414.0,129.0,72.0,75.0,155.0,...,[],[],[],阿神,good,276.0,電玩實況類,True,2020/03/09,NaN
4018,2020/04/12,3500.0,12180.0,6353.0,1170.0,1291.0,1393.0,13477.0,2612.0,3958.0,...,[],[],[],阿滴,good,269.0,知識教育類,True,2020/04/28,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a
791,2021/03/17,428.0,712.0,176.0,262.0,247.0,118.0,188.0,168.0,542.0,...,[],[],[],Joeman,good,219.0,生活搞笑類,True,2021/04/28,NaN
2372,2021/07/01,58.0,10.0,19.0,0.0,88.0,48.0,0.0,7.0,30.0,...,[],[],[],木曜4超玩,good,218.0,生活搞笑類,True,2021/07/04,NaN
2314,2020/02/19,438.0,22.0,4.0,18.0,68.0,93.0,1.0,5.0,20.0,...,[nan],[],[],木曜4超玩,good,218.0,生活搞笑類,True,2020/03/03,NaN
4414,2020/05/30,1254.0,189.0,128.0,450.0,404.0,190.0,19.0,33.0,63.0,...,[],[],[],黃氏兄弟,good,185.0,生活搞笑類,True,2020/06/03,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d
1230,2020/06/05,334.0,236.0,92.0,138.0,190.0,35.0,72.0,145.0,169.0,...,[],[],[],反骨男孩,good,156.0,生活搞笑類,True,2020/06/09,NaN


In [143]:
final_df[['Youtuber', 'sub', 'g/b', '前二後二情緒變化量', '犯錯的類別']
         ].to_csv("../sorry_event_match.csv", index=False)


KeyError: "['犯錯的類別'] not in index"

In [90]:
pd.read_csv("../sorry_event_match.csv").columns


Index(['Youtuber', 'sub', 'g/b', '前二後二情緒變化量'], dtype='object')